## Simulation of <110> dumbbell diffusion in Fe- dilute Mn alloy.
In this notebook, we use the onsager calculator we created in part 1 to compute transport coefficients for the dumbbell-mediated mechanism for dilute Mn in Fe.

The data of the energies are taken from the database given along with the paper by Messina et. al. - https://doi.org/10.1016/j.actamat.2020.03.038

We need to identify the solute-dumbbell complex states as well as the various dumbbell jump types with the appropriate labels give in this paper. We then assign appropriate state formation and dumbbell migration energies, as per the data given in the above-mentioned database.

We then compare the Green's function results with the results from SCMF calculations taken also from the above-mentioned database.

In [1]:
import numpy as np

from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
from matplotlib import pyplot as plt

from tqdm import tqdm
import pickle
import time

In [2]:
%%time
with open("FeMn_Onsg.pkl","rb") as fl:
    onsagercalculator = pickle.load(fl)

CPU times: user 10min 43s, sys: 19.4 s, total: 11min 2s
Wall time: 10min 45s


### Now, we'll look at the complex states in our thermodynamic shell
We take the first state from each symmetry group of states, and print out its info in groups
of three lines.

For each state, the number printed out in the first line is the index assgined to the
symmetry group (called the "star") the state belongs to.

The second line gives the pure dumbbell orientation vector in cartesian coordinates (Recall that the length of the orientation vector is taken to be the atomic diameter of iron).

The third line prints the position of the dumbbell with respect to the solute location
in cartesian coordinates. Recall that the lattice parameter of BCC Fe is 0.2831 nm, as per Messina et al.'s paper.

Note that the first complex state (with index 0 in the first line) is an origin state (pure dumbbell on top of solute) and is unphysical.

From index 1 onwards, we'll then have to manually match our complex states with the labels in Messina et. al. (1nnA, 1nnB etc). No interaction will be assumed when a corresponding state label is not found in Messina et. al.'s database. For example, a particualr 4nn state found below was not considered in the SCMF calculations.
The first 13 such symmetry group starting from index 1 are going to contain all the states we need to consider.

In [3]:
count = 0
for star in onsagercalculator.thermo.stars[:onsagercalculator.thermo.mixedstartindex]:
    print(count)
    db = star[0].db
    print("Dumbbell orientation: {}".format(np.round(
        onsagercalculator.pdbcontainer.iorlist[db.iorind][1], decimals=4)-0.))
    print("Dumbbell location relative to solute: {}".format(
    np.dot(onsagercalculator.crys.lattice, db.R)))
    print()
    count += 1

0
Dumbbell orientation: [-0.1782 -0.1782  0.    ]
Dumbbell location relative to solute: [0. 0. 0.]

1
Dumbbell orientation: [0.1782 0.     0.1782]
Dumbbell location relative to solute: [-0.14155  0.14155 -0.14155]

2
Dumbbell orientation: [0.1782 0.     0.1782]
Dumbbell location relative to solute: [-0.14155  0.14155  0.14155]

3
Dumbbell orientation: [-0.1782 -0.1782  0.    ]
Dumbbell location relative to solute: [0.2831 0.     0.    ]

4
Dumbbell orientation: [-0.1782 -0.1782  0.    ]
Dumbbell location relative to solute: [0.     0.     0.2831]

5
Dumbbell orientation: [-0.1782 -0.1782  0.    ]
Dumbbell location relative to solute: [0.2831 0.2831 0.    ]

6
Dumbbell orientation: [-0.1782 -0.1782  0.    ]
Dumbbell location relative to solute: [ 0.2831  0.     -0.2831]

7
Dumbbell orientation: [-0.1782 -0.1782  0.    ]
Dumbbell location relative to solute: [ 0.2831 -0.2831  0.    ]

8
Dumbbell orientation: [0.1782 0.     0.1782]
Dumbbell location relative to solute: [-0.42465 -0.14155 

In [4]:
# Next, we assign labels to the solute-dumbbell configurations.
name_to_themo_star = {"1nnA":2, "1nnB":1, "2nnA":4, "2nnB":3, "3nnA":7,"3nnB":6,"3nnC":5,
       "4nnA":10,"4nnB":11,"4nnC":8, "5nnA":12, "5nnB":13}

## Transition state energies and attempt frequencies as obtained from Messina et. al.'s database

### Pure dumbbell jumps

| Jump label | TS energy | att. freq. | jump type </br>

|J_2_KRA_dumbbell_dumbbell | -2081.1091 | 4.4447 | roto-translation, pure dumbbell </br>
|J_1_KRA_dumbbell_dumbbell | -2080.8336 | 4.4447 | rotation, pure mbbell </br>
|J_2_KRA_dumbbell_dumbbell | -2080.6599 | 4.4447 | rigid translation, pure dumbbell </br>

### Mixed dumbbell jumps
| Jump label | TS energy | att. freq. | jump type </br>

|J_2_KRA_0nnA_0nnA  | -2082.1767      |5.9297 | roto-translation, mixed dumbbell </br>
|J_2_KRA_0nnA_0nnA  | -2082.17906295  |5.9297 | rotation, mixed dumbbell </br>
|J_2_KRA_0nnA_0nnA  | -2081.8452      |5.9297 | rigid translation, mixed dumbbell </br>
|J_2_KRA_0nnA_1nnB  | -2082.0446      |4.4447 | mixed dumbbell formation-annihilation </br>

### Pure dumbbell jumps under interaction with solutes
| Jump label | TS energy | att. freq. </br>
|J_3_1nnA_2nnA | -2081.6931 | 4.4447 </br>
|J_3_1nnA_2nnB | -2081.6706 | 4.4447 </br>
|J_3_1nnA_3nnB | -2081.6771 | 4.4447 </br>
|J_3_1nnA_3nnC | -2081.6764 | 4.4447 </br>
|J_3_1nnB_2nnB | -2081.8645 | 4.4447 </br>
|J_3_1nnB_3nnB | -2081.7221 | 4.4447 </br>
|J_3_1nnB_5nnB | -2081.7316 | 4.4447 </br>
|J_3_2nnA_4nnC | -2081.5549 | 4.4447 </br>
|J_3_2nnB_4nnB | -2081.6867 | 4.4447 </br>
|J_3_2nnB_4nnC | -2081.6444 | 4.4447 </br>

In [5]:
# Indexing the jumps with the nomenclatures for different transition energies due to solute-dumbbell
# interactions.
# Note that all such jumps for whom solute dumbbell interactions affect the transition state energies
# are only 60-degree roto-translational jumps. Rigid translations are assumed to occur with the same
# transition state energy as the pure dumbbell.

jmpdict = {"1nnA_2nnA":[], "1nnA_2nnB":[], "1nnA_3nnB":[], "1nnA_3nnC":[], "1nnB_2nnB":[], "1nnB_3nnB":[],
          "1nnB_5nnB":[], "2nnA_4nnC":[], "2nnB_4nnB":[], "2nnB_4nnC":[]}
# Now identify the jumps and put them into the dictionaries
for jlistind, jlist in enumerate(onsagercalculator.jnet1):
    jmp = jlist[0]
    state1 = jmp.state1
    state2 = jmp.state2
    
    # if rigid jump, then continue
    if jmp.state1.db.iorind == jmp.state2.db.iorind:
        continue
    
    # get the indices of the symmetry groups ("star") the states belong to.
    star1 = onsagercalculator.kinetic.complexIndexdict[state1][1]
    star2 = onsagercalculator.kinetic.complexIndexdict[state2][1]
    
    # Now check if these are in the thermodynamic shell
    if star1 in onsagercalculator.thermo2kin and star2 in onsagercalculator.thermo2kin:
        thermo_star1 = onsagercalculator.thermo.complexIndexdict[state1][1]
        thermo_star2 = onsagercalculator.thermo.complexIndexdict[state2][1]
        name1 = ""
        name2 = ""
        
        # Now see which categories the states belong to
        star1found = False
        count1 = 0
        star2found = False
        count2 = 0
        for (key, value) in name_to_themo_star.items():
            if thermo_star1==value:
                star1found = True
                count1 += 1
                name1 = key
            if thermo_star2==value:
                star2found = True
                count2 += 1
                name2 = key
        # just to ensure we don't have any multiple counting business going on.
        if count1>1:
            print(thermo_star1)
        if count2>1:
            print(thermo_star2)
        # Now concatenate names
        jname = name1+"_"+name2
        jnameRev = name2+"_"+name1
        try:
            jmpdict[jname].append(jlistind)
        except:
            try:
                # maybe the jump we have is the reverse of what we stored as the label in the dictionary?
                jmpdict[jnamerev].append(jlistind)
            
            except:    
                continue

jmpdict

{'1nnA_2nnA': [7],
 '1nnA_2nnB': [5],
 '1nnA_3nnB': [1],
 '1nnA_3nnC': [6],
 '1nnB_2nnB': [4],
 '1nnB_3nnB': [3],
 '1nnB_5nnB': [2],
 '2nnA_4nnC': [9],
 '2nnB_4nnB': [8],
 '2nnB_4nnC': [10]}

In [6]:
# Next, we assign state energies to the symmetry groups.

E_f_pdb = 4.081701163
name_to_en =\
{"1nnA":-2082.04436416,"1nnB":-2082.24287998,"2nnA":-2081.93194878,"2nnB":-2082.02050066,"3nnA":-2081.87795528,
"3nnB":-2081.94900210,"3nnC":-2081.94643601,"4nnA":-2081.90793186,"4nnB":-2081.96094539,"4nnC":-2081.93724321,
"5nnA":-2081.93328589,"5nnB":-2081.95048841}

In [7]:
E_sup_pdb = -2081.44451396
E_sup_solute = -2077.71045687 
E_bulk = -2077.21734574  #E_bulk is the same as E_ref
name_to_Ef = {}
for (key, E_IB) in name_to_en.items():
    # get the binding energy first
    Eb = -E_IB + E_sup_pdb + E_sup_solute - E_bulk
    # Next, get the formation energy (relative to solute formation energy)
    name_to_Ef[key] = E_f_pdb - Eb

In [8]:
# The complex energies are set. Now, we set the mixed dumbbell energies
E_b_mdb = 2082.49273533 + E_sup_pdb + E_sup_solute - E_bulk
E_f_mdb = E_f_pdb - E_b_mdb
E_f_mdb - E_f_pdb

-0.5551102399995216

In [9]:
Jname_2_TS_en = {"1nnA_2nnA": -2081.6931, "1nnA_2nnB": -2081.6706, "1nnA_3nnB": -2081.6771,
                 "1nnA_3nnC": -2081.6764, "1nnB_2nnB": -2081.8645, "1nnB_3nnB": -2081.7221,
                 "1nnB_5nnB": -2081.7316, "2nnA_4nnC": -2081.5549, "2nnB_4nnB": -2081.6867, 
                 "2nnB_4nnC": -2081.6444}

In [11]:
# Now, we have to find the TS energies.
Jname_2_ef_ts = {}
for (key, E_IB) in Jname_2_TS_en.items():
    Eb = -E_IB + E_sup_pdb + E_sup_solute - E_bulk
    # Next, get the formation energy (relative to solute formation energy)
    Jname_2_ef_ts[key] = E_f_pdb - Eb

In [12]:
Jname_2_mig = {}
for (key, TS_en) in Jname_2_ef_ts.items():
    initstar = key[:4]
    finstar = key[5:]
    Jname_2_mig[key] = (TS_en - name_to_Ef[initstar], TS_en - name_to_Ef[finstar])

In [13]:
# omega2 and omega43 60-degree roto-translation jumps
E_IB_43, E_IB_2 = -2082.0446, -2082.1767
Eb_43, Eb_2 = -E_IB_43 + E_sup_pdb + E_sup_solute - E_bulk, -E_IB_2 + E_sup_pdb + E_sup_solute - E_bulk 
# Next, get the formation energy (relative to solute formation energy)
ef_ts_43 = E_f_pdb - Eb_43
ef_ts_2 = E_f_pdb - Eb_2
print(ef_ts_2, ef_ts_43)
print(ef_ts_2-E_f_mdb)
print(ef_ts_43 - E_f_mdb, ef_ts_43 - name_to_Ef["1nnB"])

3.8426262530004553 3.9747262530002647
0.31603532999997697
0.44813532999978634 0.19827997999982472


In [14]:
# omega2 rigid translation
E_IB_2_rigid = -2081.8452
Eb_2_rigid = -E_IB_2_rigid + E_sup_pdb + E_sup_solute - E_bulk
ef_ts_2_rigid = E_f_pdb - Eb_2_rigid
print(ef_ts_2_rigid-E_f_mdb)

0.647535329999755


In [15]:
# omega2 on-site rotation
E_IB_2_rot = -2082.1790629
Eb_2_rot = -E_IB_2_rot + E_sup_pdb + E_sup_solute - E_bulk
ef_ts_2_rot = E_f_pdb - Eb_2_rot
print(ef_ts_2_rot-E_f_mdb)

0.3136724299997695


## Mn Thermodynamic data

In [17]:
# Jump rates and energy barriers set. Now, let's set the calculations up.
vu0 = 4.4447
vu2 = 5.9297
Dconv=1e-2
# to get cm^2/s units from nm^2*THz

# pre-factors and formation energies for pure dumbbell jumps
predb0, enedb0 = np.ones(1)*np.exp(0.050), np.array([E_f_pdb])


# Here on, pre-factors are going to 1.0
# We'll measure every formation energy relative to the solute formation energy.
# so we set solute formation energy to zero, and (as per data) pre-factor to 1.
preS, eneS = np.ones(1), np.array([0.0])

# Next, interaction or the excess energies and pre-factors for solutes and dumbbells.
preSdb, eneSdb = np.ones(onsagercalculator.thermo.mixedstartindex), \
                 np.zeros(onsagercalculator.thermo.mixedstartindex)
# Now, we go over the necessary stars and assign interaction energies
for (key, index) in name_to_themo_star.items():
    eneSdb[index] = name_to_Ef[key] - E_f_pdb

predb2, enedb2 = np.ones(1), np.array([E_f_mdb])

# Transition state energies - For omega0 and omega2, the first type is the Johnson jump,
# and the second one is the rigid jump. For omega0, the third is the on-site rotation.

# Omega0 TS eneriges
# taken directly from the paper
preT0, eneT0 = Dconv*vu0*np.ones(1), np.array([E_f_pdb + 0.33541396, E_f_pdb + 0.61091396, E_f_pdb + 0.784315123])

# Omega2 TS energies
Nj2 = len(onsagercalculator.jnet2)
preT2, eneT2 = Dconv*vu2*np.ones(Nj2), np.array([ef_ts_2, ef_ts_2_rigid, ef_ts_2_rot])

# Omega43 TS energies
preT43, eneT43 = Dconv*vu0*np.ones(1), np.array([ef_ts_43])

# Omega1 TS energies
preT1 = Dconv*vu0*np.ones(len(onsagercalculator.jnet1))
eneT1 = np.array([eneT0[i] for i in onsagercalculator.om1types])
# Now, we go over the jumps that are provided and make the necessary changes
for (key, index) in jmpdict.items():
    eneT1[index] = Jname_2_ef_ts[key]
eneT1[0] = 0.0s

In [19]:
# Then we calculate the transport coefficients
from tqdm import tqdm

temp = np.arange(200, 4001, 10)

diff_aa_Mn = np.zeros(len(temp))
diff_ab_Mn = np.zeros(len(temp))
diff_bb = np.zeros(len(temp))
diff_bb_non_loc = np.zeros(len(temp))

start = time.time()
for i in tqdm(range(len(temp)), position=0, leave=True, ncols=65):
    T = temp[i]
    kT = kB*T
    bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
        onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                               preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)
    
    # get the transport coefficients
    # the uncorrelated (L_uc) and correlated (L_c) parts will be summed to get the total
    # transport coefficients.
    # "a" stands for solute, "b" for solvent (Fe)
    L0bb, (L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
    onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)
    
    L_aa = L_uc_aa + L_c_aa
    L_bb = L_uc_bb + L_c_bb
    L_ab = L_uc_ab + L_c_ab
    
    diff_aa_Mn[i] = L_aa[0][0]
    diff_ab_Mn[i] = L_ab[0][0]
    diff_bb[i] = L_bb[0][0]
    diff_bb_non_loc[i] = L0bb[0][0]
    
print(time.time() - start)

100%|██████████████████████████| 381/381 [38:31<00:00,  6.07s/it]

2311.56112575531


In [28]:
# Now let's do the infinite temeperature limit
kT = np.inf
bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
    onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                           preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)

# get the transport coefficients
L0bb, (L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab)=\
onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)

L_aa = L_uc_aa + L_c_aa
L_bb = L_uc_bb + L_c_bb
L_ab = L_uc_ab + L_c_ab

drag_inf = L_ab[0][0]/L_aa[0][0]
drag_inf

1.6652483153312445

In [31]:
# save for plotting later on
import h5py
with h5py.File("Mn_data.h5","w") as fl:
    fl.create_dataset("diff_aa", data=diff_aa_Mn)
    fl.create_dataset("diff_ab", data=diff_ab_Mn)
    fl.create_dataset("diff_bb_nl", data=diff_bb_non_loc)
    fl.create_dataset("diff_bb", data=diff_bb)
    fl.create_dataset("Temp", data=temp)
    fl.create_dataset("drag_inf", data=np.array([drag_inf]))